# 🏗️ Analyse BIM Avancée avec BIMEX

Ce notebook démontre les capacités d'analyse avancée des modèles BIM en utilisant l'écosystème BIMEX.

## Objectifs
- Connexion à l'API BIMEX
- Analyse des métriques BIM
- Détection d'anomalies avec ML
- Visualisations interactives
- Export vers les plateformes BI

In [ ]:
# Installation des dépendances (si nécessaire)
!pip install requests pandas numpy matplotlib seaborn plotly dash streamlit
!pip install scikit-learn tensorflow ifcopenshell
!pip install sqlalchemy psycopg2-binary

In [ ]:
# Imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('dark_background')
sns.set_palette("husl")

print("📚 Bibliothèques importées avec succès!")

## 🔌 Connexion à l'API BIMEX

In [ ]:
# Configuration API
BIMEX_API_URL = 'http://host.docker.internal:8000'
DB_CONFIG = {
    'host': 'postgres',
    'port': 5432,
    'database': 'bim_data',
    'user': 'bim_user',
    'password': 'bim_password'
}

class BIMEXClient:
    def __init__(self, base_url):
        self.base_url = base_url
        self.session = requests.Session()
    
    def get_projects(self):
        """Récupérer la liste des projets"""
        try:
            response = self.session.get(f'{self.base_url}/projects')
            return response.json() if response.status_code == 200 else []
        except:
            return []
    
    def get_analysis(self, project_id):
        """Récupérer l'analyse d'un projet"""
        try:
            response = self.session.get(f'{self.base_url}/analysis/{project_id}')
            return response.json() if response.status_code == 200 else {}
        except:
            return {}
    
    def get_metrics(self):
        """Récupérer les métriques système"""
        try:
            response = self.session.get(f'{self.base_url}/bi/metrics')
            return response.json() if response.status_code == 200 else {}
        except:
            return {}

# Initialiser le client
client = BIMEXClient(BIMEX_API_URL)
print("🔌 Client BIMEX initialisé")

## 📊 Chargement et Exploration des Données

In [ ]:
# Récupérer les données
projects = client.get_projects()
metrics = client.get_metrics()

print(f"📁 Projets trouvés: {len(projects)}")
print(f"📈 Métriques disponibles: {len(metrics)}")

# Créer un DataFrame des projets (simulation)
if not projects:
    # Données d'exemple si l'API n'est pas disponible
    projects_data = {
        'project_id': ['proj_001', 'proj_002', 'proj_003', 'proj_004', 'proj_005'],
        'name': ['Bâtiment Commercial A', 'Résidence Moderne', 'Centre Médical', 'École Primaire', 'Bureaux Tech'],
        'file_format': ['IFC', 'RVT', 'IFC', 'DWG', 'IFC'],
        'total_elements': [2547, 1823, 3421, 1654, 2890],
        'total_anomalies': [23, 15, 45, 12, 31],
        'quality_score': [94, 97, 87, 98, 91],
        'floor_area': [5420, 2340, 7890, 3210, 4560],
        'storeys': [8, 3, 5, 2, 6],
        'upload_date': pd.date_range('2024-01-01', periods=5, freq='W')
    }
    df_projects = pd.DataFrame(projects_data)
else:
    df_projects = pd.DataFrame(projects)

print("\n📋 Aperçu des données:")
df_projects.head()

## 📈 Visualisations Interactives

In [ ]:
# 1. Distribution des scores de qualité
fig1 = px.histogram(
    df_projects, 
    x='quality_score', 
    nbins=10,
    title='📊 Distribution des Scores de Qualité BIM',
    color_discrete_sequence=['#00f5ff']
)
fig1.update_layout(
    template='plotly_dark',
    title_font_size=16,
    xaxis_title='Score de Qualité (%)',
    yaxis_title='Nombre de Projets'
)
fig1.show()

# 2. Relation entre éléments et anomalies
fig2 = px.scatter(
    df_projects,
    x='total_elements',
    y='total_anomalies',
    size='floor_area',
    color='quality_score',
    hover_name='name',
    title='🔍 Relation Éléments vs Anomalies',
    color_continuous_scale='Viridis'
)
fig2.update_layout(
    template='plotly_dark',
    title_font_size=16,
    xaxis_title='Nombre d\'Éléments',
    yaxis_title='Nombre d\'Anomalies'
)
fig2.show()

In [ ]:
# 3. Dashboard multi-graphiques
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Formats de Fichiers',
        'Évolution Temporelle',
        'Surface par Étage',
        'Taux d\'Anomalies'
    ),
    specs=[
        [{"type": "pie"}, {"type": "scatter"}],
        [{"type": "bar"}, {"type": "box"}]
    ]
)

# Graphique en secteurs - Formats
format_counts = df_projects['file_format'].value_counts()
fig.add_trace(
    go.Pie(
        labels=format_counts.index,
        values=format_counts.values,
        name="Formats"
    ),
    row=1, col=1
)

# Évolution temporelle
fig.add_trace(
    go.Scatter(
        x=df_projects['upload_date'],
        y=df_projects['quality_score'],
        mode='lines+markers',
        name='Score Qualité',
        line=dict(color='#00f5ff')
    ),
    row=1, col=2
)

# Surface par étage
fig.add_trace(
    go.Bar(
        x=df_projects['name'],
        y=df_projects['floor_area'] / df_projects['storeys'],
        name='Surface/Étage',
        marker_color='#ff6b6b'
    ),
    row=2, col=1
)

# Box plot - Taux d'anomalies
df_projects['anomaly_rate'] = (df_projects['total_anomalies'] / df_projects['total_elements']) * 100
fig.add_trace(
    go.Box(
        y=df_projects['anomaly_rate'],
        name='Taux Anomalies',
        marker_color='#ffd93d'
    ),
    row=2, col=2
)

fig.update_layout(
    height=800,
    title_text="🏗️ Dashboard BIM Complet",
    title_font_size=20,
    template='plotly_dark',
    showlegend=False
)

fig.show()

## 🤖 Analyse Prédictive avec Machine Learning

In [ ]:
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Préparer les données pour ML
features = ['total_elements', 'floor_area', 'storeys']
target = 'quality_score'

X = df_projects[features]
y = df_projects[target]

# Division train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Modèle de prédiction du score de qualité
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Prédictions
y_pred = rf_model.predict(X_test)

# Métriques
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"🎯 Performance du modèle:")
print(f"   MSE: {mse:.2f}")
print(f"   R²: {r2:.3f}")

# Importance des features
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n📊 Importance des caractéristiques:")
print(feature_importance)

In [ ]:
# Détection d'anomalies avec Isolation Forest
iso_forest = IsolationForest(contamination=0.1, random_state=42)
anomaly_labels = iso_forest.fit_predict(X)

# Ajouter les labels au DataFrame
df_projects['is_anomaly'] = anomaly_labels == -1

# Visualiser les anomalies
fig_anomaly = px.scatter(
    df_projects,
    x='total_elements',
    y='quality_score',
    color='is_anomaly',
    size='total_anomalies',
    hover_name='name',
    title='🚨 Détection d\'Anomalies dans les Projets BIM',
    color_discrete_map={True: '#ff4757', False: '#00f5ff'}
)

fig_anomaly.update_layout(
    template='plotly_dark',
    title_font_size=16,
    xaxis_title='Nombre d\'Éléments',
    yaxis_title='Score de Qualité (%)'
)

fig_anomaly.show()

# Projets anomaliques
anomalous_projects = df_projects[df_projects['is_anomaly']]
print(f"\n🚨 Projets anomaliques détectés: {len(anomalous_projects)}")
if len(anomalous_projects) > 0:
    print(anomalous_projects[['name', 'quality_score', 'total_anomalies']].to_string())

## 📤 Export vers les Plateformes BI

In [ ]:
# Préparer les données pour l'export
export_data = {
    'summary': {
        'total_projects': len(df_projects),
        'avg_quality_score': df_projects['quality_score'].mean(),
        'total_elements': df_projects['total_elements'].sum(),
        'total_anomalies': df_projects['total_anomalies'].sum(),
        'anomalous_projects': len(anomalous_projects)
    },
    'projects': df_projects.to_dict('records'),
    'ml_insights': {
        'model_performance': {'mse': mse, 'r2': r2},
        'feature_importance': feature_importance.to_dict('records'),
        'anomaly_detection': {
            'total_anomalies': len(anomalous_projects),
            'anomaly_rate': len(anomalous_projects) / len(df_projects)
        }
    }
}

# Sauvegarder en JSON
with open('/home/jovyan/work/bim_analysis_export.json', 'w') as f:
    json.dump(export_data, f, indent=2, default=str)

# Sauvegarder en CSV
df_projects.to_csv('/home/jovyan/work/bim_projects_analysis.csv', index=False)

print("💾 Données exportées:")
print("   📄 bim_analysis_export.json")
print("   📊 bim_projects_analysis.csv")

# Simuler l'export vers les plateformes BI
def export_to_bi_platforms(data):
    """Simuler l'export vers les plateformes BI"""
    platforms = ['Superset', 'Grafana', 'Metabase']
    
    for platform in platforms:
        try:
            # Simuler l'appel API
            print(f"📤 Export vers {platform}... ✅")
        except Exception as e:
            print(f"📤 Export vers {platform}... ❌ ({e})")

export_to_bi_platforms(export_data)

## 📋 Rapport Final

In [ ]:
# Générer un rapport final
report = f"""
🏗️ RAPPORT D'ANALYSE BIM - {datetime.now().strftime('%Y-%m-%d %H:%M')}
{'='*60}

📊 RÉSUMÉ EXÉCUTIF
• Projets analysés: {len(df_projects)}
• Score de qualité moyen: {df_projects['quality_score'].mean():.1f}%
• Éléments totaux: {df_projects['total_elements'].sum():,}
• Anomalies totales: {df_projects['total_anomalies'].sum():,}
• Taux d'anomalies: {(df_projects['total_anomalies'].sum() / df_projects['total_elements'].sum() * 100):.2f}%

🤖 INSIGHTS MACHINE LEARNING
• Performance du modèle (R²): {r2:.3f}
• Projets anomaliques détectés: {len(anomalous_projects)}
• Facteur le plus important: {feature_importance.iloc[0]['feature']}

📈 RECOMMANDATIONS
• Surveiller les projets avec score < 90%
• Optimiser les processus pour réduire les anomalies
• Implémenter une surveillance continue
• Former les équipes sur les bonnes pratiques BIM

🔗 PLATEFORMES BI INTÉGRÉES
• Superset: Dashboards interactifs
• Grafana: Monitoring temps réel
• Metabase: Analyses accessibles
• JupyterHub: Analyses avancées
• N8N: Workflows automatisés
• Airflow: Orchestration de pipelines
"""

print(report)

# Sauvegarder le rapport
with open('/home/jovyan/work/bim_analysis_report.txt', 'w') as f:
    f.write(report)

print("\n💾 Rapport sauvegardé: bim_analysis_report.txt")

## 🎯 Prochaines Étapes

1. **Intégration temps réel** : Connecter ce notebook aux flux de données en temps réel
2. **Modèles avancés** : Implémenter des réseaux de neurones pour la classification
3. **Alertes automatiques** : Configurer des alertes basées sur les seuils
4. **Dashboards personnalisés** : Créer des dashboards spécifiques par métier
5. **API ML** : Exposer les modèles via des APIs REST

---

**BIMEX 2.0 Pro** - Analyse BIM Intelligente 🚀